# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

#Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#reads in csv
citi_weather = pd.read_csv("../output_data/cities.csv")
citi_weather = pd.DataFrame(citi_weather)
citi_weather = citi_weather.dropna()
del citi_weather['Unnamed: 0']
citi_weather
#note: this dataframe hasn't been cleaned of 'NaN' entries from weather.py. redo? or move the to_csv write?
# bad_city_weather= citi_weather.loc[(citi_weather['Lat']!="NaN") 
#                             | (citi_weather['Lng']!='NaN') 
#                             | (citi_weather['Max Temp']!='NaN')
#                             | (citi_weather['Cloudiness']!='NaN')
#                             | (citi_weather['Wind Speed']!='NaN')
#                             | (citi_weather['Country']!='NaN')
#                             | (citi_weather['Date']!='NaN')
#                            ]
# bad_city_weather.reset_index()
# bad_city_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,saint-philippe,-21.3585,55.7679,299.26,78.0,0.0,2.57,RE,1.612648e+09
1,havoysund,70.9963,24.6622,269.15,86.0,75.0,7.20,NO,1.612648e+09
2,east london,-33.0153,27.9116,296.88,87.0,8.0,1.10,ZA,1.612648e+09
4,hobart,-42.8794,147.3294,289.82,59.0,75.0,4.63,AU,1.612648e+09
6,cape town,-33.9258,18.4232,295.37,78.0,20.0,5.14,ZA,1.612648e+09
...,...,...,...,...,...,...,...,...,...
605,cockburn town,21.4612,-71.1419,300.15,65.0,20.0,7.20,TC,1.612648e+09
606,leh,34.1667,77.5833,258.23,51.0,0.0,0.80,IN,1.612648e+09
607,antsla,57.8256,26.5406,260.15,85.0,20.0,2.06,EE,1.612648e+09
608,hokitika,-42.7167,170.9667,291.19,73.0,4.0,1.53,NZ,1.612648e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
#Store 'Lat' and 'Lng' into locations
locations = citi_weather[['Lat', 'Lng']].astype(float)
humidity = citi_weather['Humidity'].astype(float)

In [5]:
#create a heatmap layer to map
fig = gmaps.figure()

#Add layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 1)
fig.add_layer(heat_layer)

#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#null values should already be dropped
#convert degrees from Kelvin (default) to Fahrenheit
#note: this accumulates if I run multiple times and the selection process breaks. how to reset so it always works?
citi_weather['Max Temp'] = (citi_weather['Max Temp'] - 275.13)* 9/5 + 32

#criteria: 70 < max temp < 85, cloudiness < 7, wind speed < 5
vacay_city = citi_weather.loc[(citi_weather['Max Temp'] > 70) 
                             & (citi_weather['Max Temp'] < 85)
                             & (citi_weather['Cloudiness'] < 7)
                             & (citi_weather['Wind Speed'] < 5)
                             ]
#convert back to Kelvin
citi_weather['Max Temp'] = (citi_weather['Max Temp'] - 32)* 5/9 + 275.13



### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = vacay_city.copy()

hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,saint-philippe,-21.3585,55.7679,75.434,78.0,0.0,2.57,RE,1.612648e+09,
25,hilo,19.7297,-155.0900,75.236,50.0,1.0,2.57,US,1.612648e+09,
30,vao,-22.6667,167.4833,73.670,85.0,0.0,2.52,NC,1.612648e+09,
51,kapaa,22.0752,-159.3190,70.430,49.0,1.0,2.06,US,1.612648e+09,
94,luganville,-15.5333,167.1667,79.700,68.0,3.0,2.09,VU,1.612648e+09,
122,negombo,7.2083,79.8358,73.436,83.0,0.0,1.54,LK,1.612648e+09,
139,saint-leu,-21.1500,55.2833,75.236,78.0,0.0,2.57,RE,1.612648e+09,
147,chuy,-33.6971,-53.4616,70.088,69.0,0.0,3.66,UY,1.612648e+09,
160,kavaratti,10.5669,72.6420,77.594,60.0,3.0,3.92,IN,1.612648e+09,
174,ouegoa,-20.3500,164.4333,78.746,63.0,0.0,0.71,NC,1.612648e+09,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "0, 0",
    "rankby": "distance",
    "type": "lodging",
    "locationbias": 5000,
    "keyword": "hotel",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
  
# add appropriate coordinates to params
    this_lat = row['Lat']
    this_lng = row['Lng']
    this_place = f'{this_lat},{this_lng}'
    
    params['location'] = this_place
# assemble url and make API request
    city_name = row['City']
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
#    extract results
    results = response['results']
    try:
        first_name = results[0]['name']
        print(json.dumps(first_name, indent=4))
        hotel_df.loc[index, "Hotel Name"]= first_name
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
#     try:
#         print(f"Closest hotel is {results[0]['name']}.")
        
# #         types_df.loc[index, 'name'] = results[0]['name']
# #         types_df.loc[index, 'address'] = results[0]['vicinity']
# #         types_df.loc[index, 'price_level'] = results[0]['price_level']
# #         types_df.loc[index, 'rating'] = results[0]['rating']
        
#     except (KeyError, IndexError):
#         print("Missing field/result... skipping.")
        
# print("------------")

Retrieving Results for Index 0: saint-philippe.
"Les Embruns Du Baril"
Retrieving Results for Index 25: hilo.
"Dolphin Bay Hotel"
Retrieving Results for Index 30: vao.
"OURE TERA beach resort"
Retrieving Results for Index 51: kapaa.
"Hotel Coral Reef"
Retrieving Results for Index 94: luganville.
"Santo Seaside Villas"
Retrieving Results for Index 122: negombo.
"Elsy Villa"
Retrieving Results for Index 139: saint-leu.
"Iloha Seaview Hotel"
Retrieving Results for Index 147: chuy.
"Gaboto"
Retrieving Results for Index 160: kavaratti.
"Bismillah Hotel And Lodging"
Retrieving Results for Index 174: ouegoa.
"G\u00eete Chez Acide et Nicole"
Retrieving Results for Index 179: faanui.
"Sunset Hill Lodge"
Retrieving Results for Index 194: ancud.
"Caba\u00f1as Millalobo"
Retrieving Results for Index 207: coahuayana.
"Hotel Los Arcos"
Retrieving Results for Index 217: redcliffe.
"Highland House"
Retrieving Results for Index 255: labuhan.
"OYO 1588 Hotel Bintang"
Retrieving Results for Index 256: ca

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))